# Demo implementation of a hybrid functional Petri net (HFPN)

This demo achieves the following:
- [x] Shows which packages must be imported in order to work with HFPNs.
- [x] Shows how to add places
- [x] Shows different methods for adding transitions
- [x] Demonstrates how to run the HFPN
- [x] Shows how to search for places/transitions in your petri-net via key words
Things that would be nice to add:
- [ ] An example of how to plot selected places
- [ ] An example of how to import variables/parameters from a separate script

### 1) Import HFPN backbone code (Only one time)

In [1]:
import os
import sys

# Only run this cell once to avoid confusion with directories
# Point this to the directory where HFPN.py is relative to your working directory
cwd = os.getcwd() #Get current directory
root_folder = "team-project" 
# Move to 'utils' from current directory position
sys.path.insert(0, cwd[:(cwd.index(root_folder)+len(root_folder))] + "/utils/") 

# Import HFPN class to work with hybrid functional Petri nets
from hfpn import HFPN

## Example: Simple HFPN for $2\,\text{H}_2 + \text{O}_2 \rightarrow 2 \,\text{H}_2 \text{O}$ with an artificial inhibitor

### 2) Initialize an empty HPFN

When creating the HFPN, you can specify a value for the time increment in seconds. The default value (if you don't specify a value) is 0.001 seconds, i.e. 1 millisecond. 

In [2]:
# Initialize an empty HFPN
pn = HFPN(time_step = 0.001) 

### 3) Add places to the HFPN
Add place for each of the following chemical species:
- Hydrogen H2
- Oxygen O2
- Water H2O
- Inhibitor I

Each place has 4 attributes:
- Initial number of tokens
- place_id: this uniquely identifies a place and is given by the agreed naming conventions for places
- A label that briefly describes the place
- A boolean indicating if the place is continuous or discrete

In [3]:
pn.add_place(initial_tokens=20, place_id="p_H2", label="Hydrogen", continuous=True)
pn.add_place(initial_tokens=10, place_id="p_O2", label="Oxygen", continuous=True)
pn.add_place(initial_tokens=0, place_id="p_I", label="Inhibitor", continuous=True)
pn.add_place(initial_tokens=15, place_id="p_H2O", label="Water", continuous=True)

### 4) Add transitions to the HFPN
Transitions represent reactions occurring in the biological network. So far, there are three ways to add a new transition. The difference between the three methods lies only in how a new transition is actually added by the user, the transition itself is the same for all three methods. 
- The function add_transition() is used for manual input of all reaction kinetics, e.g. rates, rate constants and stochiometric coefficients. 
- The function add_transition_with_speed_function() serves as a short simplification: here, you only need to put in one general equation (here e.g. law of mass action) for the reaction speed and the stochiometric coefficients.
- The function add_transition_with_mass_action() is written for easy implementation of reactions governed by the law of mass action. The user only needs to specify the rate constant and stochiometric coefficients and the resulting rates are calculated from the concentrations (number of tokens) of the input places. 

**Make sure you know exactly what the biological function you want to implement is so you can choose the appropriate and easiest way to introduce your transition.**

#### Method 1: add_transition
Here, we put in all functions for consumption and production speeds in manually.

In [4]:
rate_constant = 1 #rate constant
pn.add_transition(  transition_id = 't_a', 
                    label = 'Example transition Hydrogen and Oxygen to Water', 
                    input_place_ids = ['p_H2', 'p_O2', 'p_I'],
                    firing_condition = lambda a : a['p_H2'] >= 0 or a['p_O2'] >= 0 and a['p_I'] <= 0.01,
                    consumption_speed_functions = [lambda a : rate_constant * a['p_H2']**2 * a['p_O2']**1 * 2,
                                                   lambda a : rate_constant * a['p_H2']**2 * a['p_O2']**1 * 1,
                                                   lambda a : 0],
                    output_place_ids = ['p_H2O'],  
                    production_speed_functions = [lambda a : rate_constant * a['p_H2']**2 * a['p_O2']**1 * 2])
                                                   


Function arguments:
- input_place_ids: List of place_id's of all input places.
- firing condition: Define input place concentration thresholds for transition to happen. Inhibitory threshold is realized by forcing concentration of inhibitor to be very small.
- consumption_speed_functions: A list of functions for consumption of first (H2) second (O2) and third (Inhibitor) input place.
- output_place_ids: List of place_id's of all output places
- production_speed_functions: Function for production of first output place (H2O)

#### Method 2: add_transition_with_speed_function

In [10]:
rate_constant = 1
pn.add_transition_with_speed_function(
                    transition_id = 't_b',
                    label = 'Example transition b',
                    input_place_ids = ['p_H2', 'p_O2', 'p_I'],
                    firing_condition = lambda a : a['p_H2'] >= 0 or a['p_O2'] >= 0 and a['p_I'] <= 0.01,
                    reaction_speed_function = lambda a : rate_constant * a['p_H2']**2 * a['p_O2']**1,
                    consumption_coefficients = [2, 1, 0], 
                    output_place_ids = ['p_H2O'],         
                    production_coefficients = [2])      


Function arguments:
- input_place_ids: List of place_id's of all input places
- firing_condition: firing condition: Define input place concentration thresholds for transition to happen. Inhibitory threshold is realized by forcing concentration of inhibitor to be very small.
- reaction_speed_function: Function (here Mass-action) for the speed of the reaction
- consumption_coefficients: Stochiometric coefficients for consumption of input places in order (H2, O2, Inhibitor)
- output_place_ids: List of place_id's of all output places
- production_coefficients: Stochiometric coefficients for production of output places in order (H2O)

#### Method 3: add_transition_with_mass_action
Here, we put in one function for the reaction speed and the stochiometric coefficients for each input and output place.
This method can be used to easily implement reaction governed by the law of mass action. 
The user only puts in the stochiometric coefficients for all input/output places and the resulting rates 
etc are calculated in the background.

In [6]:
rate_constant = 1
pn.add_transition_with_mass_action(  transition_id = 't_c',
                    label = 'Example transition c',
                    rate_constant = rate_constant,
                    input_place_ids = ['p_H2', 'p_O2', 'p_I'], #List of place_id's of all input places
                    firing_condition = lambda a : a['p_H2'] >= 0 and a['p_O2'] >= 0 and a['p_I'] <= 0.01,
                    consumption_coefficients = [2, 1, 0], 
                    output_place_ids = ['p_H2O'],         
                    production_coefficients = [2])        


Function arguments: 
- firing_condition: Define input place concentration thresholds for transition to happen. Inhibitory threshold is realized by forcing concentration of inhibitor to be very small.
- consumption_coefficients: Stochiometric coefficients for consumption of input places in order (H2, O2, Inhibitor)
- output_place_ids: List of place_id's of all output places.
- production_coefficients: Stochiometric coefficients for production of output places in order (H2O)


### 5) Run the HFPN

When running the HFPN, you have the option to specify
- The total number of time steps for each run
- The total number of runs to be executed

In [7]:
pn.run_many_times(number_runs=1, number_time_steps=5) 
print(pn.token_storage)

Timestep 0:
	Hydrogen has 20 tokens
	Oxygen has 10 tokens
	Inhibitor has 0 tokens
	Water has 15 tokens
Timestep 1:
	Hydrogen has 9.188160356352 tokens
	Oxygen has 4.594080178176 tokens
	Inhibitor has 0.0 tokens
	Water has 25.811839643648 tokens
Timestep 2:
	Hydrogen has 7.339441473715818 tokens
	Oxygen has 3.669720736857909 tokens
	Inhibitor has 0.0 tokens
	Water has 27.66055852628418 tokens
Timestep 3:
	Hydrogen has 6.32053385493213 tokens
	Oxygen has 3.160266927466065 tokens
	Inhibitor has 0.0 tokens
	Water has 28.67946614506787 tokens
Timestep 4:
	Hydrogen has 5.644954618366221 tokens
	Oxygen has 2.8224773091831103 tokens
	Inhibitor has 0.0 tokens
	Water has 29.35504538163378 tokens
[[[ 20.          10.           0.          15.        ]
  [  9.18816036   4.59408018   0.          25.81183964]
  [  7.33944147   3.66972074   0.          27.66055853]
  [  6.32053385   3.16026693   0.          28.67946615]
  [  5.64495462   2.82247731   0.          29.35504538]
  [  5.15281667   2.57640

pn.run_many_times: Let the petri net evolve for time steps (number_time_steps). This can automatically be repeated for number_runs times.\
token.storage: A 3D matrix saving the token counts of all places at all times over each run.

### Additional functions: Searching for places/transitions via id or labels

There are functions which enable you to find all places and/or transitions that contain specified character(s):
- find_places_transitions() identifies places/transitions whose place_id/transition_id contain the specified character(s)
- find_places_transitions_labels() identifies places/transitions whose label contain the specified character(s)

In [8]:
# 1.Example: Search Id's for 'H2':
search_key = 'H2'
pn.find_places_transitions(search_key, case_sensitive = True, search_places=True, search_transitions=True)

List of all place id's containing H2:
p_H2
p_H2O

List of all transitions id's containing H2:


case_sensitive describes if the search should distinguish between upper and lower case letters.\
With search_places and search_transitions you can restrict your search to either one if desired.

In [9]:
# 2.Example: Search labels for 'Hydrogen':
search_key = 'Hydrogen'
pn.find_places_transitions_labels(search_key, case_sensitive = True, search_places=True, search_transitions=True)

List of all place labels containing Hydrogen:
Hydrogen

List of all transitions labels containing Hydrogen:
Example transition Hydrogen and Oxygen to Water


Function inputs:
- case_sensitive describes if the search should distinguish between upper and lower case letters.
- search_places and search_transitions: you can restrict your search to either one if desired.